# Graph Calculation

In [ ]:
import branca.colormap as cm
import pandas as pd
import geopandas as gpd
import networkx as nx
import folium
import osmnx as ox

In [ ]:
class DictSmallest(dict):
    def __setitem__(self, key, value):
        if (key not in self) or (key in self and self[key] > value):
            dict.__setitem__(self, key, value)
    def update(self, dict):
        for key, value in dict.items():
            if (key not in self) or (key in self and self[key] > value):
                self[key] =  value

In [ ]:
all_station_df = pd.read_csv('cleaned_stations.csv')
all_sub_df = all_station_df.loc[(all_station_df["subway"] == True) | (all_station_df["suburban"] == True)]

In [ ]:
G = ox.graph_from_address("Hamburg, Germany", 
  dist=10000, 
  network_type="walk", 
  simplify=True)
gdf_nodes = ox.graph_to_gdfs(G, edges=False)

In [ ]:
list_of_poi = []
for index, row in all_sub_df.iterrows():
    list_of_poi.append(ox.distance.nearest_nodes(G, Y=row.lat, X=row.lon))

G = ox.project_graph(G)

node_distances = DictSmallest()

for poi in list_of_poi:
    tmp_res = nx.shortest_path_length(G, source=poi)
    node_distances.update(tmp_res)
 

coords = {key: {"lon": G.nodes[key]["lon"], "lat": G.nodes[key]["lat"], "distance": node_distances[key]} for key in list(G.nodes())}

In [ ]:
node_distances = DictSmallest()

for poi in list_of_poi:
    tmp_res = nx.shortest_path_length(G, source=poi)
    node_distances.update(tmp_res)

# Visualization

In [ ]:
trip_times = range(1, 51, 1)

iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0.3, return_hex=True)
iso_colors.reverse()

colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, 51, 5))
    

def color_mapping_function(val, iso_colors):

    for time, color in zip(trip_times, iso_colors):
        if val < time :
            return color

    return iso_colors[-1]

coords = {key: {"x": G.nodes[key]["lon"], "y": G.nodes[key]["lat"], "color": color_mapping_function(node_distances[key], iso_colors)} for key in list(G.nodes())}

In [ ]:
colormap = cm.LinearColormap(colors=iso_colors)
colormap = colormap.to_step(index=range(0, 51, 5))

m = folium.Map(
    location=[53.555, 9.9914],
    zoom_start=12,
    prefer_canvas=True,
)


for val in coords.values():
    folium.Circle(
        location=[val["y"], val["x"]],
        radius=50,
        stroke=False,
        fill=True,
        color=val["color"],
        fill_opacity=0.3,
        interactive=True,
    ).add_to(m)
colormap.add_to(m)
m